# <font color=deeppink>**Combining All Data Files into One DataFrame**</font>

## <font color=royalblue>Load Necessary Libraries</font>

In [1]:
import pandas as pd
import numpy as np
import datetime
import csv

## <font color=royalblue>Pertinent Functions</font>

#### <font color=darkviolet>Functions for Combining the Datasets</font>

In [2]:
# Determine if the Student Is of Multiple Races
def multiRace(HispanicFlag, AmerIndFlag, AsianFlag, BlackFlag, WhiteFlag, HawaiianFlag):
    ## For This Categorical Variable, 0 Indicates "No" and 1 Indicates "Yes"
    if sum((HispanicFlag, AmerIndFlag, AsianFlag, BlackFlag, WhiteFlag, HawaiianFlag)) >= 2:
        multinational = 1
    else:
        multinational = 0
    
    return multinational

# Extract the Desired Text from the String Entry
def extract_value(column_value):
    words = str(column_value).split()

    # Extract the Relevant Words from the List
    relevant_words = words[1:words.index("Name:")]

    return_value = ""
    
    # Create a New String of Just the Relevant Words
    for word in relevant_words:
        return_value = return_value + word + " "
        
    # Remove the Trailing Space
    return_value = return_value.strip()
    
    return return_value

# Combine the Information of All the Academies to Compute the Entire School's Statistics
def get_academies_info(df, ref_school_code, ref_address):
    # Create a List of All the Extracted and Computed Information
    info_list = [] 
    
    
    # Use the Information of One of the Academies
    ## District
    info_list.append(extract_value(df[df.NCES_SchoolID == ref_school_code].District))
    
    ## Full Address
    info_list.append(extract_value(df[df.NCES_SchoolID == ref_school_code].HS_FullAddress))
    
    ## Latitude
    info_list.append(float(df[df.NCES_SchoolID == ref_school_code].HS_Lat))
    
    ## Longitude
    info_list.append(float(df[df.NCES_SchoolID == ref_school_code].HS_Long))
    
    ## ZIP
    info_list.append(extract_value(df[df.NCES_SchoolID == ref_school_code].HS_ZIP))

    ## Phone
    info_list.append(extract_value(df[df.NCES_SchoolID == ref_school_code].Phone))
    
    ## Charter School Flag
    ### Must Be an Integer
    info_list.append(int(df[df.NCES_SchoolID == ref_school_code].Charter))
    
    ## Magnet School Flag
    ### Must Be an Integer
    info_list.append(int(df[df.NCES_SchoolID == ref_school_code].Magnet))
    
    
                     
    # Use the Information of All Four Academies to Compute the Averages
    ## Number of Students
    ### Must Be a Float
    info_list.append(float(df[df.HS_FullAddress == ref_address].NumStudents.mean()))
    
    ## Number of Teachers
    ### Must Be a Float
    info_list.append(float(df[df.HS_FullAddress == ref_address].NumTeachers.mean()))
                 
    ## Student:Teacher Ratio
    ### Must Be a Float
    info_list.append(float(df[df.HS_FullAddress == ref_address].StudentTeacherRatio.mean()))
    
    ## Number of Students with Free Lunch
    ### Must Be a Float
    info_list.append(float(df[df.HS_FullAddress == ref_address].NumFreeLunch.mean()))
    
    ## Number of Students with Reduced Lunch
    ### Must Be a Float
    info_list.append(float(df[df.HS_FullAddress == ref_address].NumReducedLunch.mean()))
   
    return info_list

# Differentiate between Low, Average, and High Crime Rates
def categorize_crime(index_value):
    # Index Values of 100 Represent Average Crime for the Country
    if index_value == 100:
        return "Average"
    
    # Values More than 100 Represent Higher Crime than the National Average
    elif index_value > 100:
        return "High"
    
    # Values Less than 100 Represent Lower Crime than the National Average
    else:
        return "Low"

#### <font color=darkviolet>Functions for Computing the Distances</font>

A location's 3D coordinates are $\begin{bmatrix} 
    \,R\sin(\lambda)\cos(\theta)\, \\ 
    \,R\cos(\lambda)\cos(\theta)\, \\
    \,R\sin(\theta)\,
\end{bmatrix},$ where $R$ is the radius of the Earth ($\approx 6,367.5$ km)$,$ $\lambda$ is the longitude$,$ and $\theta$ is the latitude.

The distance between two points $a$ and $b$ along the surface of the Earth is $R\measuredangle(a,b),$ where $\measuredangle \,(a,b) = \arccos \left( \frac{a^{T}b}{\parallel a \parallel \parallel b \parallel} \right).$

In [3]:
# Express the Location in 3D Coordinates Using Latitude and Longitude in Radians
def make_3D_Coordinates(latitude, longitude):
    # Define R
    R = 6367.5
    
    # Compute the Coordinate Values
    coordinates = np.array([R * np.sin(longitude) * np.cos(latitude),
                            R * np.cos(longitude) * np.cos(latitude),
                            R * np.sin(latitude)])
    
    return coordinates
    
# Compute the Distance Along the Surface of the Earth
def get_distance(homeLat, homeLong, schoolLat, schoolLong):
    # Define R
    R = 6367.5
    
    # Convert the Latitudes and Longitudes to Radians
    homeLat_rad = homeLat * (np.pi/180)
    homeLong_rad = homeLong * (np.pi/180)
    schoolLat_rad = schoolLat * (np.pi/180)
    schoolLong_rad = schoolLong * (np.pi/180)
    
    # Create an Array of the 3D Coordinates for Each Location
    homeCoord = make_3D_Coordinates(homeLat_rad, homeLong_rad)
    schoolCoord = make_3D_Coordinates(schoolLat_rad, schoolLong_rad)
    
    # Compute the Angle between the Student's Home and School
    angle_between = np.arccos(homeCoord @ schoolCoord / (np.linalg.norm(homeCoord) * np.linalg.norm(schoolCoord)))
    
    # Compute the Distance
    distance = R * angle_between
    
    # The Distance Along the Surface of the Earth Between the Student's Home and School (in Kilometers)
    return distance

## <font color=royalblue>Read In Data</font>

#### <font color=darkviolet>Upward Bound</font>

##### <font color=mediumorchid>Student Data</font>

In [4]:
UB_data = pd.read_csv("/content/drive/MyDrive/Thesis/Codes/Combining Datasets/ub_data.csv")
UB_data

,ID,DOB,Gender,Hispanic,AmerInd,Asian,Black,White,Hawaiian,LimitedEnglish,...,PS_SchoolID,PS_GradeLevel,Cert_or_Diploma,Cert_Date,AssocDeg,AssocDeg_Date,BachDeg,BachDeg_Date,PS_Graduated,STEMDeg
0,0,1989-11-08,1,0,0,1,0,0,0,1,...,#009344,5,2,"(2222, 22, 22)",2,"(2222, 22, 22)",1,"(2012, 5, 16)",1,1
1,1,1990-10-28,1,0,0,0,1,0,0,0,...,#002629,7,2,"(2222, 22, 22)",2,"(2222, 22, 22)",7,"(7777, 77, 77)",0,0
2,2,1990-12-07,1,1,0,0,0,0,0,0,...,#001397,5,2,"(2222, 22, 22)",2,"(2222, 22, 22)",1,"(2012, 5, 15)",1,-1
3,3,1989-12-27,1,0,0,0,1,0,0,0,...,#007502,5,2,"(2222, 22, 22)",2,"(2222, 22, 22)",1,"(2013, 1, 13)",9,9
4,4,1990-04-10,1,0,0,0,1,0,0,0,...,#002629,3,2,"(2222, 22, 22)",2,"(2222, 22, 22)",6,"(6666, 66, 66)",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,142,1996-08-21,0,0,0,0,1,0,0,0,...,#002617,7,2,"(2222, 22, 22)",7,"(7777, 77, 77)",7,"(7777, 77, 77)",0,0
143,143,1997-05-07,1,1,0,0,0,1,0,0,...,#002629,5,2,"(2222, 22, 22)",2,"(2222, 22, 22)",1,"(2021, 5, 1)",1,1
144,144,1997-05-02,0,1,0,0,0,1,0,0,...,#009344,5,2,"(2222, 22, 22)",2,"(2222, 22, 22)",1,"(2019, 12, 21)",1,0
145,145,1997-06-19,1,1,0,0,0,0,0,0,...,#009344,7,2,"(2222, 22, 22)",7,"(7777, 77, 77)",7,"(7777, 77, 77)",0,0


In [5]:
UB_data.dtypes

ID                       int64
DOB                     object
Gender                   int64
Hispanic                 int64
AmerInd                  int64
Asian                    int64
Black                    int64
White                    int64
Hawaiian                 int64
LimitedEnglish           int64
Eligibility              int64
AcademNeed               int64
Disconnected             int64
HS_SchoolID              int64
Date_EnteredUB          object
Grade_EnteredUB          int64
Participation            int64
StartGradeLevel          int64
EndGradeLevel            int64
GPA                    float64
GradStatus               int64
GraduationDate          object
HSGrad_Age               int64
AcademAch_ELA            int64
AcademAch_Math           int64
Employed                 int64
CulturalAct              int64
CommServ                 int64
LEPServs                 int64
Student_FullAddress     object
Student_ZIP             object
Absences                 int64
SAT_Read

##### <font color=mediumorchid>Upward Bound Latitudes and Longitudes</font>

In [6]:
ubLatLong_df = pd.read_csv("/content/drive/MyDrive/Thesis/Codes/Combining Datasets/ub_latLongs.csv")

##### <font color=lightsalmon>*Merge with `UB_data`*</font>

In [7]:
UB_df = pd.merge(UB_data, ubLatLong_df, how="inner", left_on="ID", right_on="ID")

##### <font color=darkturquoise>*Modify the Format of Columns, as Needed*</font>

In [8]:
# Remove the Hash Mark from the Front of the Zip Codes and Save Keep the Column as a String Datatype
UB_df["Student_ZIP"] = UB_df["Student_ZIP"].transform(lambda code: code[1:])

# Remove the Hash Mark from the Front of the Post Secondary Codes and Save Keep the Column as a String Datatype
UB_df["PS_SchoolID"] = UB_df["PS_SchoolID"].transform(lambda code: code[1:])

# Create a Column for the Multinational Flag
UB_df["Multinational"] = UB_df.apply(lambda x: multiRace(x.Hispanic, x.AmerInd, x.Asian, x.Black, x.White, x.Hawaiian), axis=1)

# Remove the ID, DOB_Match, and FullAddress Columns
## These Are No Longer Needed and/or Are Redundant with DOB and Student_FullAddress
UB_df.drop(["ID", "DOB_Match","FullAddress"], axis=1, inplace=True)

# Reorder the DataFrame's Columns
UB_df = UB_df.reindex(columns=["DOB", "Gender", "Hispanic", "AmerInd", "Asian", "Black", "White", "Hawaiian", "Multinational",
                               "LimitedEnglish", "Eligibility", "AcademNeed", "Disconnected", "HS_SchoolID", "Date_EnteredUB", 
                               "Grade_EnteredUB", "Participation", "StartGradeLevel", "EndGradeLevel", "GPA", "GradStatus", 
                               "GraduationDate", "HSGrad_Age", "AcademAch_ELA", "AcademAch_Math", "Employed", "CulturalAct", 
                               "CommServ", "LEPServs", "Student_FullAddress", "Student_Lat", "Student_Long", "Student_ZIP", 
                               "Absences", "SAT_Reading", "SAT_Math", "SAT_Writing", "AP", "Honors", "PS_DateEnrolled", 
                               "PS_Cohort", "PS_SchoolID", "PS_GradeLevel", "Cert_or_Diploma", "Cert_Date", "AssocDeg", 
                               "AssocDeg_Date", "BachDeg", "BachDeg_Date", "PS_Graduated", "STEMDeg"])

#### <font color=darkviolet>High School Information</font>

In [9]:
# Column Names in the File
hs_files_columns = ["NCES School ID", "School Name", "District", "Street Address", "City", "State", "ZIP", "Phone", "Charter", 
                    "Magnet*", "Title I School*", "Title 1 School Wide*", "Students*", "Teachers*", "Student Teacher Ratio*", 
                    "Free Lunch*", "Reduced Lunch*"]

# Column Name Revisions
new_hs_column_names = {"NCES School ID": "NCES_SchoolID", 
                       "School Name": "HS_SchoolName", 
                       "Street Address": "HS_StreetAddress",
                       "City": "HS_City",
                       "State": "HS_State", 
                       "ZIP": "HS_ZIP",
                       "Magnet*": "Magnet", 
                       "Title I School*": "Title1_School", 
                       "Title 1 School Wide*": "Title1_SchoolWide", 
                       "Students*": "NumStudents", 
                       "Teachers*": "NumTeachers", 
                       "Student Teacher Ratio*": "StudentTeacherRatio", 
                       "Free Lunch*": "NumFreeLunch", 
                       "Reduced Lunch*": "NumReducedLunch"}

##### <font color=mediumorchid>Paterson</font>

In [10]:
paterson_schools_info = pd.read_excel("/content/drive/MyDrive/Thesis/Codes/Combining Datasets/paterson_ncesdata.xlsx", usecols=hs_files_columns, skiprows=12)
paterson_schools_info

,NCES School ID,School Name,District,Street Address,City,State,ZIP,Phone,Charter,Magnet*,Title I School*,Title 1 School Wide*,Students*,Teachers*,Student Teacher Ratio*,Free Lunch*,Reduced Lunch*
0,341269003473,14# Avenue Early Learning Center,Paterson Public School District,660 14th Avenue,Paterson,NJ,7504,(973)321-0660,No,†,No,†,77.0,–,†,35.0,0.0
1,341269000353,Alexander Hamilton Academy,Paterson Public School District,11 27 16TH AVENUE,PATERSON,NJ,7501,(973)321-0320,No,†,Yes,Yes,525.0,34.00000,15.4000000,314.0,16.0
2,341269003111,Alternative High School,Paterson Public School District,350 MARKET STREET,PATERSON,NJ,7501,(973)321-0570,No,†,Yes,Yes,126.0,26.00000,4.8000000,70.0,0.0
3,341269004890,Charles J. Riley School 9,Paterson Public School District,6 TIMOTHY STREET,PATERSON,NJ,7503,(973)321-0090,No,†,Yes,Yes,736.0,65.00000,11.3000000,602.0,43.0
4,341269004870,Dale Avenue,Paterson Public School District,21 Dale Ave,PATERSON,NJ,7501,(973)321-0410,No,†,Yes,Yes,243.0,30.00000,8.1000000,130.0,3.0
5,341269003263,Don Bosco Academy,Paterson Public School District,764 11TH AVENUE,PATERSON,NJ,7514,(973)321-0580,No,†,Yes,Yes,833.0,51.00000,16.3000000,488.0,1.0
6,341269003382,Dr. Hani Awadallah School,Paterson Public School District,515 MARSHALL STREET,PATERSON,NJ,7503,(973)321-1000,No,†,Yes,Yes,598.0,50.50000,11.8000000,335.0,0.0
7,341269005916,Dr. Martin Luther King Jr. Educational Complex,Paterson Public School District,851 E 28TH STREET,PATERSON,NJ,7513,(973)321-0300,No,†,Yes,Yes,603.0,59.50000,10.1000000,326.0,0.0
8,341269004872,Edward W. Kilpatrick,Paterson Public School District,295-315 ELLISON STREET,PATERSON,NJ,7501,(973)321-0330,No,†,Yes,Yes,347.0,31.00000,11.2000000,263.0,0.0
9,341269003370,Garrett Morgan Academy,Paterson Public School District,200 Grand Street,Paterson,NJ,7501,(973)321-2540,No,†,No,†,183.0,–,†,94.0,0.0


##### <font color=mediumorchid>Newark</font>

In [11]:
newark_schools_info = pd.read_excel("/content/drive/MyDrive/Thesis/Codes/Combining Datasets/newark_ncesdata.xlsx", usecols=hs_files_columns, skiprows=12)
newark_schools_info

,NCES School ID,School Name,District,Street Address,City,State,ZIP,Phone,Charter,Magnet*,Title I School*,Title 1 School Wide*,Students*,Teachers*,Student Teacher Ratio*,Free Lunch*,Reduced Lunch*
0,341134002222,Abington Avenue School,Newark Public School District,209 ABINGTON AVENUE,NEWARK,NJ,7107,(973)268-5230,No,†,Yes,Yes,879.0,56.00000,15.7000000,662.00000,53.00000
1,341134000791,American History High School,Newark Public School District,74 Montgomery St,NEWARK,NJ,7103,(973)733-7333,No,†,Yes,Yes,445.0,41.50000,10.7000000,305.00000,36.00000
2,341134002228,Ann Street School,Newark Public School District,30 ANN ST,NEWARK,NJ,7105,(973)465-4890,No,†,Yes,Yes,1243.0,83.00000,15.0000000,774.00000,142.00000
3,341134002188,Arts High School,Newark Public School District,550 M L KING BLVD,NEWARK,NJ,7102,(973)733-7391,No,†,Yes,Yes,614.0,54.00000,11.4000000,354.00000,75.00000
4,341134002232,Avon Avenue School,Newark Public School District,219 AVON AVE,NEWARK,NJ,7108,(973)733-6750,No,†,Yes,Yes,551.0,42.00000,13.1000000,461.00000,20.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,341134002202,University High School,Newark Public School District,55 CLINTON PL,NEWARK,NJ,7108,(973)374-3190,No,†,Yes,Yes,486.0,43.00000,11.3000000,315.00000,25.00000
59,341134003117,UPLIFT Academy,Newark Public School District,150 Newton St,NEWARK,NJ,7103,(973)424-4391,No,†,Yes,Yes,3.0,–,†,†,†
60,341134002206,Weequahic High School,Newark Public School District,279 CHANCELLOR AVE,NEWARK,NJ,7112,(973)705-3923,No,†,Yes,Yes,385.0,37.00000,10.4000000,207.00000,9.00000
61,341134002208,West Side High School,Newark Public School District,403 South Orange Avenue,Newark,NJ,7103,(973)733-7019,No,†,Yes,No,620.0,44.00000,14.1000000,354.00000,35.00000


##### <font color=darkturquoise>*Stack the Districts' DataFrames One on Top of the Other*</font>

In [12]:
# New DataFrame is:
## df =
## [Paterson DataFrame
##  Newark DataFrame]
PatersonAndNewark = pd.concat([paterson_schools_info, newark_schools_info], axis=0, ignore_index=True)
PatersonAndNewark

,NCES School ID,School Name,District,Street Address,City,State,ZIP,Phone,Charter,Magnet*,Title I School*,Title 1 School Wide*,Students*,Teachers*,Student Teacher Ratio*,Free Lunch*,Reduced Lunch*
0,341269003473,14# Avenue Early Learning Center,Paterson Public School District,660 14th Avenue,Paterson,NJ,7504,(973)321-0660,No,†,No,†,77.0,–,†,35.0,0.0
1,341269000353,Alexander Hamilton Academy,Paterson Public School District,11 27 16TH AVENUE,PATERSON,NJ,7501,(973)321-0320,No,†,Yes,Yes,525.0,34.00000,15.4000000,314.0,16.0
2,341269003111,Alternative High School,Paterson Public School District,350 MARKET STREET,PATERSON,NJ,7501,(973)321-0570,No,†,Yes,Yes,126.0,26.00000,4.8000000,70.0,0.0
3,341269004890,Charles J. Riley School 9,Paterson Public School District,6 TIMOTHY STREET,PATERSON,NJ,7503,(973)321-0090,No,†,Yes,Yes,736.0,65.00000,11.3000000,602.0,43.0
4,341269004870,Dale Avenue,Paterson Public School District,21 Dale Ave,PATERSON,NJ,7501,(973)321-0410,No,†,Yes,Yes,243.0,30.00000,8.1000000,130.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,341134002202,University High School,Newark Public School District,55 CLINTON PL,NEWARK,NJ,7108,(973)374-3190,No,†,Yes,Yes,486.0,43.00000,11.3000000,315.00000,25.00000
109,341134003117,UPLIFT Academy,Newark Public School District,150 Newton St,NEWARK,NJ,7103,(973)424-4391,No,†,Yes,Yes,3.0,–,†,†,†
110,341134002206,Weequahic High School,Newark Public School District,279 CHANCELLOR AVE,NEWARK,NJ,7112,(973)705-3923,No,†,Yes,Yes,385.0,37.00000,10.4000000,207.00000,9.00000
111,341134002208,West Side High School,Newark Public School District,403 South Orange Avenue,Newark,NJ,7103,(973)733-7019,No,†,Yes,No,620.0,44.00000,14.1000000,354.00000,35.00000


##### <font color=mediumorchid>Passaic Tech</font>

In [13]:
passaicTech_schools_info = pd.read_excel("/content/drive/MyDrive/Thesis/Codes/Combining Datasets/passaictech_ncesdata.xlsx", usecols=hs_files_columns, skiprows=12)
passaicTech_schools_info

,NCES School ID,School Name,District,Street Address,City,State,ZIP,Phone,Charter,Magnet*,Title I School*,Title 1 School Wide*,Students*,Teachers*,Student Teacher Ratio*,Free Lunch*,Reduced Lunch*
0,341263004860,Passaic County Technical Institute,Passaic County Vocational School District,45 REINHARDT ROAD,WAYNE,NJ,7470,(973)790-6000,No,†,Yes,No,3429.0,289.0,11.9,1663.0,304.0


##### <font color=darkturquoise>*Stack the Districts' DataFrames One on Top of the Other*</font>

In [14]:
# New DataFrame is:
## df =
## [Paterson DataFrame
##  Newark DataFrame
##  Passaic DataFrame]
K12Schools = pd.concat([PatersonAndNewark, passaicTech_schools_info], axis=0, ignore_index=True)
K12Schools

,NCES School ID,School Name,District,Street Address,City,State,ZIP,Phone,Charter,Magnet*,Title I School*,Title 1 School Wide*,Students*,Teachers*,Student Teacher Ratio*,Free Lunch*,Reduced Lunch*
0,341269003473,14# Avenue Early Learning Center,Paterson Public School District,660 14th Avenue,Paterson,NJ,7504,(973)321-0660,No,†,No,†,77.0,–,†,35.0,0.0
1,341269000353,Alexander Hamilton Academy,Paterson Public School District,11 27 16TH AVENUE,PATERSON,NJ,7501,(973)321-0320,No,†,Yes,Yes,525.0,34.00000,15.4000000,314.0,16.0
2,341269003111,Alternative High School,Paterson Public School District,350 MARKET STREET,PATERSON,NJ,7501,(973)321-0570,No,†,Yes,Yes,126.0,26.00000,4.8000000,70.0,0.0
3,341269004890,Charles J. Riley School 9,Paterson Public School District,6 TIMOTHY STREET,PATERSON,NJ,7503,(973)321-0090,No,†,Yes,Yes,736.0,65.00000,11.3000000,602.0,43.0
4,341269004870,Dale Avenue,Paterson Public School District,21 Dale Ave,PATERSON,NJ,7501,(973)321-0410,No,†,Yes,Yes,243.0,30.00000,8.1000000,130.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,341134003117,UPLIFT Academy,Newark Public School District,150 Newton St,NEWARK,NJ,7103,(973)424-4391,No,†,Yes,Yes,3.0,–,†,†,†
110,341134002206,Weequahic High School,Newark Public School District,279 CHANCELLOR AVE,NEWARK,NJ,7112,(973)705-3923,No,†,Yes,Yes,385.0,37.00000,10.4000000,207.00000,9.00000
111,341134002208,West Side High School,Newark Public School District,403 South Orange Avenue,Newark,NJ,7103,(973)733-7019,No,†,Yes,No,620.0,44.00000,14.1000000,354.00000,35.00000
112,341134002368,Wilson Avenue School,Newark Public School District,19 WILSON AVE,NEWARK,NJ,7108,(973)465-4910,No,†,Yes,Yes,1135.0,66.00000,17.2000000,772.00000,76.00000


##### <font color=darkturquoise>*Modify the Format of Columns, as Needed*</font>

In [15]:
# Rename the Columns
K12Schools.rename(columns=new_hs_column_names, inplace=True)

# Fix the Zip Code
## Convert the Zip Code to a String and Add a Zero to the Front
K12Schools["HS_ZIP"] = K12Schools["HS_ZIP"].transform(lambda x: str("0" + str(x)))

# Make the Values Categorical
## Yes, No, Not Appliable, Unknown
K12Schools.replace(["Yes", "No", "†", "–"], [1, 0, 9, -1], inplace=True)

# Create a String of the School's Entire Address
K12Schools["HS_FullAddress"] = K12Schools.HS_StreetAddress + ", " + K12Schools.HS_City + ", " + K12Schools.HS_State

# Remove the HS_StreetAddress, HS_City, and HS_State Columns
## These Are No Longer Needed and Are Redundant with HS_FullAddress
K12Schools.drop(["HS_StreetAddress", "HS_City", "HS_State"], axis=1, inplace=True)

# Convert the Index into a Column
K12Schools.reset_index(inplace=True)

K12Schools

,index,NCES_SchoolID,HS_SchoolName,District,HS_ZIP,Phone,Charter,Magnet,Title1_School,Title1_SchoolWide,NumStudents,NumTeachers,StudentTeacherRatio,NumFreeLunch,NumReducedLunch,HS_FullAddress
0,0,341269003473,14# Avenue Early Learning Center,Paterson Public School District,07504,(973)321-0660,0,9,0,9,77.0,-1,9,35.0,0.0,"660 14th Avenue, Paterson, NJ"
1,1,341269000353,Alexander Hamilton Academy,Paterson Public School District,07501,(973)321-0320,0,9,1,1,525.0,34.00000,15.4000000,314.0,16.0,"11 27 16TH AVENUE, PATERSON, NJ"
2,2,341269003111,Alternative High School,Paterson Public School District,07501,(973)321-0570,0,9,1,1,126.0,26.00000,4.8000000,70.0,0.0,"350 MARKET STREET, PATERSON, NJ"
3,3,341269004890,Charles J. Riley School 9,Paterson Public School District,07503,(973)321-0090,0,9,1,1,736.0,65.00000,11.3000000,602.0,43.0,"6 TIMOTHY STREET, PATERSON, NJ"
4,4,341269004870,Dale Avenue,Paterson Public School District,07501,(973)321-0410,0,9,1,1,243.0,30.00000,8.1000000,130.0,3.0,"21 Dale Ave, PATERSON, NJ"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,341134003117,UPLIFT Academy,Newark Public School District,07103,(973)424-4391,0,9,1,1,3.0,-1,9,9,9,"150 Newton St, NEWARK, NJ"
110,110,341134002206,Weequahic High School,Newark Public School District,07112,(973)705-3923,0,9,1,1,385.0,37.00000,10.4000000,207.00000,9.00000,"279 CHANCELLOR AVE, NEWARK, NJ"
111,111,341134002208,West Side High School,Newark Public School District,07103,(973)733-7019,0,9,1,0,620.0,44.00000,14.1000000,354.00000,35.00000,"403 South Orange Avenue, Newark, NJ"
112,112,341134002368,Wilson Avenue School,Newark Public School District,07108,(973)465-4910,0,9,1,1,1135.0,66.00000,17.2000000,772.00000,76.00000,"19 WILSON AVE, NEWARK, NJ"


##### <font color=mediumorchid>High School Latitudes and Longitudes</font>

In [16]:
HSLatLong_df = pd.read_csv("/content/drive/MyDrive/Thesis/Codes/Combining Datasets/HS_latLongs.csv", usecols=["FullAddress", "HS_Lat", "HS_Long"])

# Convert the Index into a Column
HSLatLong_df.reset_index(inplace=True)

HSLatLong_df

,index,FullAddress,HS_Lat,HS_Long
0,0,"660 14th Avenue, Paterson, NJ",40.917022,-74.149923
1,1,"11 27 16TH AVENUE, PATERSON, NJ",40.916208,-74.166842
2,2,"350 MARKET STREET, PATERSON, NJ",40.914211,-74.163861
3,3,"6 TIMOTHY STREET, PATERSON, NJ",40.895381,-74.157852
4,4,"21 Dale Ave, PATERSON, NJ",40.913956,-74.168631
...,...,...,...,...
109,109,"150 Newton St, NEWARK, NJ",40.737695,-74.189605
110,110,"279 CHANCELLOR AVE, NEWARK, NJ",40.709304,-74.220042
111,111,"403 South Orange Avenue, Newark, NJ",40.741702,-74.202375
112,112,"19 WILSON AVE, NEWARK, NJ",40.727576,-74.154320


##### <font color=lightsalmon>*Merge with `K12Schools`*</font>

In [17]:
SecSchools = pd.merge(K12Schools, HSLatLong_df, how="inner", left_on="index", right_on="index")
SecSchools

,index,NCES_SchoolID,HS_SchoolName,District,HS_ZIP,Phone,Charter,Magnet,Title1_School,Title1_SchoolWide,NumStudents,NumTeachers,StudentTeacherRatio,NumFreeLunch,NumReducedLunch,HS_FullAddress,FullAddress,HS_Lat,HS_Long
0,0,341269003473,14# Avenue Early Learning Center,Paterson Public School District,07504,(973)321-0660,0,9,0,9,77.0,-1,9,35.0,0.0,"660 14th Avenue, Paterson, NJ","660 14th Avenue, Paterson, NJ",40.917022,-74.149923
1,1,341269000353,Alexander Hamilton Academy,Paterson Public School District,07501,(973)321-0320,0,9,1,1,525.0,34.00000,15.4000000,314.0,16.0,"11 27 16TH AVENUE, PATERSON, NJ","11 27 16TH AVENUE, PATERSON, NJ",40.916208,-74.166842
2,2,341269003111,Alternative High School,Paterson Public School District,07501,(973)321-0570,0,9,1,1,126.0,26.00000,4.8000000,70.0,0.0,"350 MARKET STREET, PATERSON, NJ","350 MARKET STREET, PATERSON, NJ",40.914211,-74.163861
3,3,341269004890,Charles J. Riley School 9,Paterson Public School District,07503,(973)321-0090,0,9,1,1,736.0,65.00000,11.3000000,602.0,43.0,"6 TIMOTHY STREET, PATERSON, NJ","6 TIMOTHY STREET, PATERSON, NJ",40.895381,-74.157852
4,4,341269004870,Dale Avenue,Paterson Public School District,07501,(973)321-0410,0,9,1,1,243.0,30.00000,8.1000000,130.0,3.0,"21 Dale Ave, PATERSON, NJ","21 Dale Ave, PATERSON, NJ",40.913956,-74.168631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,341134003117,UPLIFT Academy,Newark Public School District,07103,(973)424-4391,0,9,1,1,3.0,-1,9,9,9,"150 Newton St, NEWARK, NJ","150 Newton St, NEWARK, NJ",40.737695,-74.189605
110,110,341134002206,Weequahic High School,Newark Public School District,07112,(973)705-3923,0,9,1,1,385.0,37.00000,10.4000000,207.00000,9.00000,"279 CHANCELLOR AVE, NEWARK, NJ","279 CHANCELLOR AVE, NEWARK, NJ",40.709304,-74.220042
111,111,341134002208,West Side High School,Newark Public School District,07103,(973)733-7019,0,9,1,0,620.0,44.00000,14.1000000,354.00000,35.00000,"403 South Orange Avenue, Newark, NJ","403 South Orange Avenue, Newark, NJ",40.741702,-74.202375
112,112,341134002368,Wilson Avenue School,Newark Public School District,07108,(973)465-4910,0,9,1,1,1135.0,66.00000,17.2000000,772.00000,76.00000,"19 WILSON AVE, NEWARK, NJ","19 WILSON AVE, NEWARK, NJ",40.727576,-74.154320


##### <font color=darkturquoise>*Modify the Format of Columns, as Needed*</font>

In [18]:
# Remove the index and FullAddress Columns
## These Are No Longer Needed and/or Are Redundant with HS_FullAddress
SecSchools.drop(["index", "FullAddress"], axis=1, inplace=True)

# Reorder the DataFrame's Columns
SecSchools = SecSchools.reindex(columns=["NCES_SchoolID", "HS_SchoolName", "District", "HS_FullAddress", "HS_Lat", 
                                         "HS_Long", "HS_ZIP", "Phone", "Charter", "Magnet", "Title1_School", 
                                         "Title1_SchoolWide", "NumStudents", "NumTeachers", "StudentTeacherRatio", 
                                         "NumFreeLunch", "NumReducedLunch"])

SecSchools

,NCES_SchoolID,HS_SchoolName,District,HS_FullAddress,HS_Lat,HS_Long,HS_ZIP,Phone,Charter,Magnet,Title1_School,Title1_SchoolWide,NumStudents,NumTeachers,StudentTeacherRatio,NumFreeLunch,NumReducedLunch
0,341269003473,14# Avenue Early Learning Center,Paterson Public School District,"660 14th Avenue, Paterson, NJ",40.917022,-74.149923,07504,(973)321-0660,0,9,0,9,77.0,-1,9,35.0,0.0
1,341269000353,Alexander Hamilton Academy,Paterson Public School District,"11 27 16TH AVENUE, PATERSON, NJ",40.916208,-74.166842,07501,(973)321-0320,0,9,1,1,525.0,34.00000,15.4000000,314.0,16.0
2,341269003111,Alternative High School,Paterson Public School District,"350 MARKET STREET, PATERSON, NJ",40.914211,-74.163861,07501,(973)321-0570,0,9,1,1,126.0,26.00000,4.8000000,70.0,0.0
3,341269004890,Charles J. Riley School 9,Paterson Public School District,"6 TIMOTHY STREET, PATERSON, NJ",40.895381,-74.157852,07503,(973)321-0090,0,9,1,1,736.0,65.00000,11.3000000,602.0,43.0
4,341269004870,Dale Avenue,Paterson Public School District,"21 Dale Ave, PATERSON, NJ",40.913956,-74.168631,07501,(973)321-0410,0,9,1,1,243.0,30.00000,8.1000000,130.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,341134003117,UPLIFT Academy,Newark Public School District,"150 Newton St, NEWARK, NJ",40.737695,-74.189605,07103,(973)424-4391,0,9,1,1,3.0,-1,9,9,9
110,341134002206,Weequahic High School,Newark Public School District,"279 CHANCELLOR AVE, NEWARK, NJ",40.709304,-74.220042,07112,(973)705-3923,0,9,1,1,385.0,37.00000,10.4000000,207.00000,9.00000
111,341134002208,West Side High School,Newark Public School District,"403 South Orange Avenue, Newark, NJ",40.741702,-74.202375,07103,(973)733-7019,0,9,1,0,620.0,44.00000,14.1000000,354.00000,35.00000
112,341134002368,Wilson Avenue School,Newark Public School District,"19 WILSON AVE, NEWARK, NJ",40.727576,-74.154320,07108,(973)465-4910,0,9,1,1,1135.0,66.00000,17.2000000,772.00000,76.00000


In [19]:
SecSchools.dtypes

NCES_SchoolID            int64
HS_SchoolName           object
District                object
HS_FullAddress          object
HS_Lat                 float64
HS_Long                float64
HS_ZIP                  object
Phone                   object
Charter                  int64
Magnet                   int64
Title1_School            int64
Title1_SchoolWide        int64
NumStudents            float64
NumTeachers             object
StudentTeacherRatio     object
NumFreeLunch            object
NumReducedLunch         object
dtype: object

##### <font color=darkturquoise>*Convert the Datatypes of Columns, as Needed*</font>

In [20]:
dtype_conversions_dict = {"NumTeachers": float, 
                          "StudentTeacherRatio": float, 
                          "NumFreeLunch": float, 
                          "NumReducedLunch": float}

SecSchools = SecSchools.astype(dtype_conversions_dict)

# Verify the Column Types Were Effectively Modified
SecSchools.dtypes

NCES_SchoolID            int64
HS_SchoolName           object
District                object
HS_FullAddress          object
HS_Lat                 float64
HS_Long                float64
HS_ZIP                  object
Phone                   object
Charter                  int64
Magnet                   int64
Title1_School            int64
Title1_SchoolWide        int64
NumStudents            float64
NumTeachers            float64
StudentTeacherRatio    float64
NumFreeLunch           float64
NumReducedLunch        float64
dtype: object

##### <font color=mediumorchid>Create a Row for John F. Kennedy High School</font>

In [21]:
# Pass the JFK DataFrame and the Necessary Reference Values
jfk_info = get_academies_info(SecSchools[SecSchools.HS_FullAddress == "61-127 PREAKNESS AVENUE, PATERSON, NJ"],
                              ## (School of Architecture and Construction Trades)
                              341269003171,
                              ## Same Address for All Academies
                              "61-127 PREAKNESS AVENUE, PATERSON, NJ")

jfk_dict = {"NCES_SchoolID": 341269004864,
            "HS_SchoolName": "John F. Kennedy High School",
            "District": jfk_info[0],
            "HS_FullAddress": jfk_info[1],
            "HS_Lat": jfk_info[2],
            "HS_Long": jfk_info[3],
            "HS_ZIP": jfk_info[4],
            "Phone": jfk_info[5],
            "Charter": jfk_info[6],
            "Magnet": jfk_info[7],
            "Title1_School": 1,
            "Title1_SchoolWide": 0,
            "NumStudents": jfk_info[8],
            "NumTeachers": jfk_info[9], 
            "StudentTeacherRatio": jfk_info[10],
            "NumFreeLunch": jfk_info[11],
            "NumReducedLunch": jfk_info[12]}

print(jfk_dict)

SecSchools = SecSchools.append(jfk_dict, ignore_index=True)

{'NCES_SchoolID': 341269004864, 'HS_SchoolName': 'John F. Kennedy High School', 'District': 'Paterson Public School District', 'HS_FullAddress': '61-127 PREAKNESS AVENUE, PATERSON, NJ', 'HS_Lat': 40.9145333, 'HS_Long': -74.1872983, 'HS_ZIP': '07502', 'Phone': '(973)321-0500', 'Charter': 0, 'Magnet': 9, 'Title1_School': 1, 'Title1_SchoolWide': 0, 'NumStudents': 575.25, 'NumTeachers': 42.875, 'StudentTeacherRatio': 13.450000000000001, 'NumFreeLunch': 310.75, 'NumReducedLunch': 0.0}


##### <font color=mediumorchid>Create a Row for Eastside High School</font>

In [22]:
# Pass the Eastside DataFrame and the Necessary Reference Values
eastside_info = get_academies_info(SecSchools[SecSchools.HS_FullAddress == "150 PARK AVENUE, PATERSON, NJ"],
                              ## (School Of Culinary Arts Hospitality and Tourism)
                              341269003103,
                              ## Same Address for All Academies
                              "150 PARK AVENUE, PATERSON, NJ")

eastside_dict = {"NCES_SchoolID": 341269004866, 
                 "HS_SchoolName": "Eastside High School",
                 "District" : eastside_info[0],
                 "HS_FullAddress": eastside_info[1],
                 "HS_Lat": eastside_info[2],
                 "HS_Long": eastside_info[3],
                 "HS_ZIP": eastside_info[4],
                 "Phone":  eastside_info[5],
                 "Charter": eastside_info[6],
                 "Magnet": eastside_info[7],
                 "Title1_School": 1,
                 "Title1_SchoolWide": 0,
                 "NumStudents": eastside_info[8],
                 "NumTeachers": eastside_info[9], 
                 "StudentTeacherRatio": eastside_info[10],
                 "NumFreeLunch": eastside_info[11],
                 "NumReducedLunch": eastside_info[12]}

print(eastside_dict)

SecSchools = SecSchools.append(eastside_dict, ignore_index=True)

{'NCES_SchoolID': 341269004866, 'HS_SchoolName': 'Eastside High School', 'District': 'Paterson Public School District', 'HS_FullAddress': '150 PARK AVENUE, PATERSON, NJ', 'HS_Lat': 40.9143052, 'HS_Long': -74.15987, 'HS_ZIP': '07501', 'Phone': '(973)321-2489', 'Charter': 0, 'Magnet': 9, 'Title1_School': 1, 'Title1_SchoolWide': 0, 'NumStudents': 668.3333333333334, 'NumTeachers': 49.0, 'StudentTeacherRatio': 13.633333333333333, 'NumFreeLunch': 361.0, 'NumReducedLunch': 2.6666666666666665}


In [23]:
SecSchools

,NCES_SchoolID,HS_SchoolName,District,HS_FullAddress,HS_Lat,HS_Long,HS_ZIP,Phone,Charter,Magnet,Title1_School,Title1_SchoolWide,NumStudents,NumTeachers,StudentTeacherRatio,NumFreeLunch,NumReducedLunch
0,341269003473,14# Avenue Early Learning Center,Paterson Public School District,"660 14th Avenue, Paterson, NJ",40.917022,-74.149923,07504,(973)321-0660,0,9,0,9,77.000000,-1.000,9.000000,35.00,0.000000
1,341269000353,Alexander Hamilton Academy,Paterson Public School District,"11 27 16TH AVENUE, PATERSON, NJ",40.916208,-74.166842,07501,(973)321-0320,0,9,1,1,525.000000,34.000,15.400000,314.00,16.000000
2,341269003111,Alternative High School,Paterson Public School District,"350 MARKET STREET, PATERSON, NJ",40.914211,-74.163861,07501,(973)321-0570,0,9,1,1,126.000000,26.000,4.800000,70.00,0.000000
3,341269004890,Charles J. Riley School 9,Paterson Public School District,"6 TIMOTHY STREET, PATERSON, NJ",40.895381,-74.157852,07503,(973)321-0090,0,9,1,1,736.000000,65.000,11.300000,602.00,43.000000
4,341269004870,Dale Avenue,Paterson Public School District,"21 Dale Ave, PATERSON, NJ",40.913956,-74.168631,07501,(973)321-0410,0,9,1,1,243.000000,30.000,8.100000,130.00,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,341134002208,West Side High School,Newark Public School District,"403 South Orange Avenue, Newark, NJ",40.741702,-74.202375,07103,(973)733-7019,0,9,1,0,620.000000,44.000,14.100000,354.00,35.000000
112,341134002368,Wilson Avenue School,Newark Public School District,"19 WILSON AVE, NEWARK, NJ",40.727576,-74.154320,07108,(973)465-4910,0,9,1,1,1135.000000,66.000,17.200000,772.00,76.000000
113,341263004860,Passaic County Technical Institute,Passaic County Vocational School District,"45 REINHARDT ROAD, WAYNE, NJ",40.931466,-74.204267,07470,(973)790-6000,0,9,1,0,3429.000000,289.000,11.900000,1663.00,304.000000
114,341269004864,John F. Kennedy High School,Paterson Public School District,"61-127 PREAKNESS AVENUE, PATERSON, NJ",40.914533,-74.187298,07502,(973)321-0500,0,9,1,0,575.250000,42.875,13.450000,310.75,0.000000


#### <font color=darkorange>Merge with the Upward Bound Data</font>

In [24]:
UB_SecSchools = pd.merge(UB_df, SecSchools, how="left", left_on="HS_SchoolID", right_on="NCES_SchoolID")

##### <font color=darkturquoise>*Check for Missing Values*</font>

In [25]:
print("Size of DataFrame: ", UB_SecSchools.shape)
print("Size of DataFrame after Dropping Null Values: ", UB_SecSchools.dropna().shape)

Size of DataFrame:  (147, 68)
Size of DataFrame after Dropping Null Values:  (147, 68)


In [26]:
# Remove the NCES_SchoolID Column
## This is No Longer Needed and is Redundant with HS_SchoolID
UB_SecSchools.drop("NCES_SchoolID", axis=1, inplace=True)

#### <font color=darkviolet>Department of Education (DOE) Data</font>

In [27]:
# Column Names in the Files
student_group_columns = ["SchoolName", "Female", "Male", "Economically Disadvantaged Students", "Students with Disabilities", 
                         "English Learners", "Homeless Students", "Students in Foster Care"]

racial_group_columns = ["SchoolName", "White", "Hispanic", "Black or African American", "Asian",
                        "Native Hawaiian or Pacific Islander", "American Indian or Alaska Native"]

# Column Name Revisions
new_doe_column_names = {"Female": "%_Female", 
                        "Male": "%_Male", 
                        "Economically Disadvantaged Students": "%_EconomDisadv", 
                        "Students with Disabilities": "%_w/Disabilities", 
                        "English Learners": "%_ELL", 
                        "Homeless Students": "%_Homeless", 
                        "Students in Foster Care": "%_FosterCare",
                        "White": "%_White", 
                        "Hispanic": "%_Hispanic", 
                        "Black or African American": "%_Black", 
                        "Asian": "%_Asian",
                        "Native Hawaiian or Pacific Islander": "%_Hawaiian", 
                        "American Indian or Alaska Native": "%_AmericanIndian"}

In [28]:
DOEData_StudentGroup = pd.read_excel("/content/drive/MyDrive/Thesis/Codes/Combining Datasets/Database_SchoolDetail.xlsx", 
                                     sheet_name="EnrollmentTrendsByStudentGroup", usecols=student_group_columns)
DOEData_StudentGroup

,SchoolName,Female,Male,Economically Disadvantaged Students,Students with Disabilities,English Learners,Homeless Students,Students in Foster Care
0,Emma C Attales,48,52,48.2,12.2,5.4,0.8,0.5
1,H Ashton Marsh,49.5,50.5,45.9,17.2,8.6,0.2,0.9
2,Atlantic City High School,45,55,77.3,16,9.7,1.7,0.3
3,Sovereign Avenue School,51,49,94.3,14.9,22.1,1.2,0.3
4,Chelsea Heights School,49,51,91,7.5,9.6,0.5,0
...,...,...,...,...,...,...,...,...
2504,Union County TEAMS Charter School,55.5,44.5,93.7,7,3.7,5.2,0
2505,Unity Charter School,46,53.5,14,20.3,1.3,0,0
2506,University Academy Charter High School,55.5,44.5,69,17.3,0,0,0.2
2507,University Heights Charter School,49.5,50.5,98.8,8.7,4.9,0,0.3


In [29]:
DOEData_RacialEthnic = pd.read_excel("/content/drive/MyDrive/Thesis/Codes/Combining Datasets/Database_SchoolDetail.xlsx", 
                                     sheet_name="EnrollmentByRacialEthnicGroup", usecols=racial_group_columns)
DOEData_RacialEthnic

,SchoolName,White,Hispanic,Black or African American,Asian,Native Hawaiian or Pacific Islander,American Indian or Alaska Native
0,Emma C Attales,50.3,23.1,16.3,4.1,0,0
1,H Ashton Marsh,53.4,20.8,13.7,7.3,0.2,0
2,Atlantic City High School,12.6,37,27,21.6,1.2,0
3,Sovereign Avenue School,1.9,69.3,4.8,22,1.4,0
4,Chelsea Heights School,3.9,48.6,17.8,27.4,0.5,0.3
...,...,...,...,...,...,...,...
2504,Union County TEAMS Charter School,0.3,37.9,60.8,1,0,0
2505,Unity Charter School,52.1,11.9,14.8,14,0,0
2506,University Academy Charter High School,3.5,30,57.2,9.2,0,0
2507,University Heights Charter School,0,16.8,82.9,0.1,0,0


##### <font color=darkturquoise>*Combine the Two Sheets of Information*</font>

In [30]:
DOE_Data = pd.merge(DOEData_StudentGroup, DOEData_RacialEthnic, how="inner", on="SchoolName")
DOE_Data

,SchoolName,Female,Male,Economically Disadvantaged Students,Students with Disabilities,English Learners,Homeless Students,Students in Foster Care,White,Hispanic,Black or African American,Asian,Native Hawaiian or Pacific Islander,American Indian or Alaska Native
0,Emma C Attales,48,52,48.2,12.2,5.4,0.8,0.5,50.3,23.1,16.3,4.1,0,0
1,H Ashton Marsh,49.5,50.5,45.9,17.2,8.6,0.2,0.9,53.4,20.8,13.7,7.3,0.2,0
2,Atlantic City High School,45,55,77.3,16,9.7,1.7,0.3,12.6,37,27,21.6,1.2,0
3,Sovereign Avenue School,51,49,94.3,14.9,22.1,1.2,0.3,1.9,69.3,4.8,22,1.4,0
4,Chelsea Heights School,49,51,91,7.5,9.6,0.5,0,3.9,48.6,17.8,27.4,0.5,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3372,Union County TEAMS Charter School,55.5,44.5,93.7,7,3.7,5.2,0,0.3,37.9,60.8,1,0,0
3373,Unity Charter School,46,53.5,14,20.3,1.3,0,0,52.1,11.9,14.8,14,0,0
3374,University Academy Charter High School,55.5,44.5,69,17.3,0,0,0.2,3.5,30,57.2,9.2,0,0
3375,University Heights Charter School,49.5,50.5,98.8,8.7,4.9,0,0.3,0,16.8,82.9,0.1,0,0


##### <font color=darkturquoise>*Modify the Format of Columns, as Needed*</font>

In [31]:
# Rename the Columns
DOE_Data.rename(columns=new_doe_column_names, inplace=True)
DOE_Data

,SchoolName,%_Female,%_Male,%_EconomDisadv,%_w/Disabilities,%_ELL,%_Homeless,%_FosterCare,%_White,%_Hispanic,%_Black,%_Asian,%_Hawaiian,%_AmericanIndian
0,Emma C Attales,48,52,48.2,12.2,5.4,0.8,0.5,50.3,23.1,16.3,4.1,0,0
1,H Ashton Marsh,49.5,50.5,45.9,17.2,8.6,0.2,0.9,53.4,20.8,13.7,7.3,0.2,0
2,Atlantic City High School,45,55,77.3,16,9.7,1.7,0.3,12.6,37,27,21.6,1.2,0
3,Sovereign Avenue School,51,49,94.3,14.9,22.1,1.2,0.3,1.9,69.3,4.8,22,1.4,0
4,Chelsea Heights School,49,51,91,7.5,9.6,0.5,0,3.9,48.6,17.8,27.4,0.5,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3372,Union County TEAMS Charter School,55.5,44.5,93.7,7,3.7,5.2,0,0.3,37.9,60.8,1,0,0
3373,Unity Charter School,46,53.5,14,20.3,1.3,0,0,52.1,11.9,14.8,14,0,0
3374,University Academy Charter High School,55.5,44.5,69,17.3,0,0,0.2,3.5,30,57.2,9.2,0,0
3375,University Heights Charter School,49.5,50.5,98.8,8.7,4.9,0,0.3,0,16.8,82.9,0.1,0,0


#### <font color=darkorange>Merge with `UB_SecSchools`</font>

In [32]:
UB_SecSchools_wDOE = pd.merge(UB_SecSchools, DOE_Data, how="left", left_on="HS_SchoolName", right_on="SchoolName")

##### <font color=darkturquoise>*Check for Missing Values*</font>

In [33]:
print("Size of DataFrame: ", UB_SecSchools_wDOE.shape)
print("Size of DataFrame after Dropping Null Values: ", UB_SecSchools_wDOE.dropna().shape)

Size of DataFrame:  (147, 81)
Size of DataFrame after Dropping Null Values:  (147, 81)


In [34]:
# Remove the SchoolName Column
## This is No Longer Needed and is Redundant with HS_SchoolName
UB_SecSchools_wDOE.drop("SchoolName", axis=1, inplace=True)

#### <font color=darkviolet>College Information</font>

In [35]:
# Column Names in the File
college_file_columns = ["SchoolCode", "SchoolName", "Address", "City", "StateCode", "ZipCode"]


# Column Name Revisions
new_college_column_names = {"SchoolCode": "PS_SchoolCode", 
                            "SchoolName": "PS_SchoolName", 
                            "Address": "PS_Address",
                            "City": "PS_City",
                            "StateCode": "PS_State", 
                            "ZipCode": "PS_ZIP"}

##### <font color=mediumorchid>Federal School Codes</font>

In [36]:
# Read in the File
colleges = pd.read_excel("/content/drive/MyDrive/Thesis/Codes/Combining Datasets/FedSchoolCodeList.xlsx", usecols=college_file_columns)

##### <font color=mediumorchid>United States' State and Region Codes</font>

In [37]:
# Read in the File of USA State Abbreviations
state_codes = pd.read_excel("/content/drive/MyDrive/Thesis/Codes/Combining Datasets/US_StateCodes.xlsx")
state_codes

,State or Region Code,Name
0,AA,Armed Forces America
1,AE,Armed Forces
2,AK,Alaska
3,AL,Alabama
4,AP,Armed Forces Pacific
5,AR,Arkansas
6,AZ,Arizona
7,CA,California
8,CO,Colorado
9,CT,Connecticut


#### <font color=darkorange>Merge with `colleges`</font>

In [38]:
# Only Use Rows with Colleges in the United States
UScolleges = pd.merge(colleges, state_codes, how="inner", left_on="StateCode", right_on="State or Region Code")
UScolleges

,SchoolCode,SchoolName,Address,City,StateCode,ZipCode,State or Region Code,Name
0,B04724,WIDENER UNIVERSITY - DELAWARE LAW SCHOOL,4601 CONCORD PIKE,WILMINGTON,DE,19803,DE,Delaware
1,001428,DELAWARE STATE UNIVERSITY,1200 NORTH DUPONT HIGHWAY,DOVER,DE,19901,DE,Delaware
2,001429,GOLDEY BEACOM COLLEGE,4701 LIMESTONE RD,WILMINGTON,DE,19808,DE,Delaware
3,001431,UNIVERSITY OF DELAWARE,224 HULLIHEN HALL,NEWARK,DE,19716,DE,Delaware
4,001433,WESLEY COLLEGE,120 N STATE STREET,DOVER,DE,19901,DE,Delaware
...,...,...,...,...,...,...,...,...
5830,003935,UNIV OF GUAM,UOG STATION,MANGILAO,GU,96923,GU,Guam
5831,015361,GUAM COMMUNITY COLLEGE,P.O. BOX 23069,BARRIGADA,GU,96921,GU,Guam
5832,034383,PACIFIC ISLANDS UNIVERSITY,172 KINNEY'S ROAD,MANGILAO,GU,96913,GU,Guam
5833,003946,UNIVERSITY OF THE VIRGIN ISLANDS,8266 JOHN BREWERS BAY,ST. THOMAS,VI,00802,VI,Virgin Islands


##### <font color=darkturquoise>*Modify the Format of Columns, as Needed*</font>

In [39]:
# Remove the State or Region Code and Name Columns
## Only Used to Mask the Initial DataFrame
UScolleges.drop(["State or Region Code", "Name"], axis=1, inplace=True)

# Rename the Columns
UScolleges.rename(columns=new_college_column_names, inplace=True)

# Create a String of the School's Entire Address
UScolleges["PS_FullAddress"] = UScolleges.PS_Address + ", " + UScolleges.PS_City + ", " + UScolleges.PS_State

# Remove the PS_Address, PS_City, and PS_State Columns
## These Are No Longer Needed and Are Redundant with PS_FullAddress
UScolleges.drop(["PS_Address", "PS_City", "PS_State"], axis=1, inplace=True)

# Convert the Index into a Column
UScolleges.reset_index(inplace=True)

UScolleges

,index,PS_SchoolCode,PS_SchoolName,PS_ZIP,PS_FullAddress
0,0,B04724,WIDENER UNIVERSITY - DELAWARE LAW SCHOOL,19803,"4601 CONCORD PIKE, WILMINGTON, DE"
1,1,001428,DELAWARE STATE UNIVERSITY,19901,"1200 NORTH DUPONT HIGHWAY, DOVER, DE"
2,2,001429,GOLDEY BEACOM COLLEGE,19808,"4701 LIMESTONE RD, WILMINGTON, DE"
3,3,001431,UNIVERSITY OF DELAWARE,19716,"224 HULLIHEN HALL, NEWARK, DE"
4,4,001433,WESLEY COLLEGE,19901,"120 N STATE STREET, DOVER, DE"
...,...,...,...,...,...
5830,5830,003935,UNIV OF GUAM,96923,"UOG STATION, MANGILAO, GU"
5831,5831,015361,GUAM COMMUNITY COLLEGE,96921,"P.O. BOX 23069, BARRIGADA, GU"
5832,5832,034383,PACIFIC ISLANDS UNIVERSITY,96913,"172 KINNEY'S ROAD, MANGILAO, GU"
5833,5833,003946,UNIVERSITY OF THE VIRGIN ISLANDS,00802,"8266 JOHN BREWERS BAY, ST. THOMAS, VI"


##### <font color=mediumorchid>Colleges Latitudes and Longitudes</font>

In [40]:
PSLatLong_df = pd.read_csv("/content/drive/MyDrive/Thesis/Codes/Combining Datasets/PS_latLongs.csv", usecols=["FullAddress", "PS_Lat", "PS_Long"])

# Convert the Index into a Column
PSLatLong_df.reset_index(inplace=True)

PSLatLong_df

,index,FullAddress,PS_Lat,PS_Long
0,0,"4601 CONCORD PIKE, WILMINGTON, DE",39.817422,-75.546321
1,1,"1200 NORTH DUPONT HIGHWAY, DOVER, DE",39.188338,-75.539764
2,2,"4701 LIMESTONE RD, WILMINGTON, DE",39.741471,-75.689670
3,3,"224 HULLIHEN HALL, NEWARK, DE",39.679565,-75.753085
4,4,"120 N STATE STREET, DOVER, DE",39.162611,-75.528034
...,...,...,...,...
5830,5830,"UOG STATION, MANGILAO, GU",13.432211,144.803322
5831,5831,"P.O. BOX 23069, BARRIGADA, GU",13.470766,144.818130
5832,5832,"172 KINNEY'S ROAD, MANGILAO, GU",13.470212,144.845622
5833,5833,"8266 JOHN BREWERS BAY, ST. THOMAS, VI",18.342633,-64.980834


##### <font color=lightsalmon>*Merge with `UScolleges`*</font>

In [41]:
colleges_info = pd.merge(UScolleges, PSLatLong_df, how="inner", left_on="index", right_on="index")
colleges_info

,index,PS_SchoolCode,PS_SchoolName,PS_ZIP,PS_FullAddress,FullAddress,PS_Lat,PS_Long
0,0,B04724,WIDENER UNIVERSITY - DELAWARE LAW SCHOOL,19803,"4601 CONCORD PIKE, WILMINGTON, DE","4601 CONCORD PIKE, WILMINGTON, DE",39.817422,-75.546321
1,1,001428,DELAWARE STATE UNIVERSITY,19901,"1200 NORTH DUPONT HIGHWAY, DOVER, DE","1200 NORTH DUPONT HIGHWAY, DOVER, DE",39.188338,-75.539764
2,2,001429,GOLDEY BEACOM COLLEGE,19808,"4701 LIMESTONE RD, WILMINGTON, DE","4701 LIMESTONE RD, WILMINGTON, DE",39.741471,-75.689670
3,3,001431,UNIVERSITY OF DELAWARE,19716,"224 HULLIHEN HALL, NEWARK, DE","224 HULLIHEN HALL, NEWARK, DE",39.679565,-75.753085
4,4,001433,WESLEY COLLEGE,19901,"120 N STATE STREET, DOVER, DE","120 N STATE STREET, DOVER, DE",39.162611,-75.528034
...,...,...,...,...,...,...,...,...
5830,5830,003935,UNIV OF GUAM,96923,"UOG STATION, MANGILAO, GU","UOG STATION, MANGILAO, GU",13.432211,144.803322
5831,5831,015361,GUAM COMMUNITY COLLEGE,96921,"P.O. BOX 23069, BARRIGADA, GU","P.O. BOX 23069, BARRIGADA, GU",13.470766,144.818130
5832,5832,034383,PACIFIC ISLANDS UNIVERSITY,96913,"172 KINNEY'S ROAD, MANGILAO, GU","172 KINNEY'S ROAD, MANGILAO, GU",13.470212,144.845622
5833,5833,003946,UNIVERSITY OF THE VIRGIN ISLANDS,00802,"8266 JOHN BREWERS BAY, ST. THOMAS, VI","8266 JOHN BREWERS BAY, ST. THOMAS, VI",18.342633,-64.980834


##### <font color=darkturquoise>*Modify the Format of Columns, as Needed*</font>

In [42]:
# Remove the index and FullAddress Columns
## These Are No Longer Needed and/or Are Redundant with PS_FullAddress
colleges_info.drop(["index", "FullAddress"], axis=1, inplace=True)

# Reorder the DataFrame's Columns
colleges_info = colleges_info.reindex(columns=["PS_SchoolCode", "PS_SchoolName", "PS_FullAddress", "PS_Lat", 
                                               "PS_Long", "PS_ZIP"])

colleges_info

,PS_SchoolCode,PS_SchoolName,PS_FullAddress,PS_Lat,PS_Long,PS_ZIP
0,B04724,WIDENER UNIVERSITY - DELAWARE LAW SCHOOL,"4601 CONCORD PIKE, WILMINGTON, DE",39.817422,-75.546321,19803
1,001428,DELAWARE STATE UNIVERSITY,"1200 NORTH DUPONT HIGHWAY, DOVER, DE",39.188338,-75.539764,19901
2,001429,GOLDEY BEACOM COLLEGE,"4701 LIMESTONE RD, WILMINGTON, DE",39.741471,-75.689670,19808
3,001431,UNIVERSITY OF DELAWARE,"224 HULLIHEN HALL, NEWARK, DE",39.679565,-75.753085,19716
4,001433,WESLEY COLLEGE,"120 N STATE STREET, DOVER, DE",39.162611,-75.528034,19901
...,...,...,...,...,...,...
5830,003935,UNIV OF GUAM,"UOG STATION, MANGILAO, GU",13.432211,144.803322,96923
5831,015361,GUAM COMMUNITY COLLEGE,"P.O. BOX 23069, BARRIGADA, GU",13.470766,144.818130,96921
5832,034383,PACIFIC ISLANDS UNIVERSITY,"172 KINNEY'S ROAD, MANGILAO, GU",13.470212,144.845622,96913
5833,003946,UNIVERSITY OF THE VIRGIN ISLANDS,"8266 JOHN BREWERS BAY, ST. THOMAS, VI",18.342633,-64.980834,00802


#### <font color=darkorange>Merge with `UB_SecSchools_wDOE`</font>

In [43]:
UB_AllSchools = pd.merge(UB_SecSchools_wDOE, colleges_info, how="left", left_on="PS_SchoolID", right_on="PS_SchoolCode")

##### <font color=darkturquoise>*Check for Missing Values*</font>

In [44]:
print("Size of DataFrame: ", UB_AllSchools.shape)
print("Size of DataFrame after Dropping Null Values: ", UB_AllSchools.dropna().shape)

Size of DataFrame:  (147, 86)
Size of DataFrame after Dropping Null Values:  (146, 86)


##### <font color=darkturquoise>*Check for Rows That Couldn't Merge*</font>

In [45]:
UB_AllSchools[UB_AllSchools.PS_SchoolCode.isnull()].shape

(1, 86)

In [46]:
# Drop the Student with All "Not Applicable" College Data
UB_AllSchools.dropna(inplace=True)

##### <font color=darkturquoise>*Check for Missing Values*</font>

In [47]:
print("Size of DataFrame: ", UB_AllSchools.shape)
print("Size of DataFrame after Dropping Null Values: ", UB_AllSchools.dropna().shape)

Size of DataFrame:  (146, 86)
Size of DataFrame after Dropping Null Values:  (146, 86)


In [48]:
# Remove the PS_SchoolCode Column
## This is No Longer Needed and is Redundant with PS_SchoolID
UB_AllSchools.drop("PS_SchoolCode", axis=1, inplace=True)

#### <font color=darkviolet>Crime Information</font>

In [49]:
# Column Names in the File
crime_file_columns = ["ID", "CRMCYTOTC","CRMCYPERC", "CRMCYMURD", "CRMCYRAPE", "CRMCYROBB", "CRMCYASST", "CRMCYPROC", 
                      "CRMCYBURG", "CRMCYLARC", "CRMCYMVEH", "TOTPOP_CY", "Shape_Area"]

# Column Name Revisions
hs_crime_column_names = {"ID": "HS_Crime_ZIP", 
                         "CRMCYTOTC": "HS_CrimesIndex_2020",
                         "CRMCYPERC": "HS_PersonalCrimesIndex_2020", 
                         "CRMCYMURD": "HS_MurdersIndex_2020", 
                         "CRMCYRAPE": "HS_RapesIndex_2020", 
                         "CRMCYROBB": "HS_RobberiesIndex_2020", 
                         "CRMCYASST": "HS_AssaultsIndex_2020",
                         "CRMCYPROC": "HS_PropertyCrimesIndex_2020", 
                         "CRMCYBURG": "HS_BurglariesIndex_2020",
                         "CRMCYLARC": "HS_LarceniesIndex_2020", 
                         "CRMCYMVEH": "HS_VehicleTheftsIndex_2020",
                         "TOTPOP_CY": "HS_AreaPopulation_2020",
                         "Shape_Area": "HS_Area"}

student_crime_column_names = {"ID": "Student_Crime_ZIP", 
                              "CRMCYTOTC": "Student_CrimesIndex_2020",
                              "CRMCYPERC": "Student_PersonalCrimesIndex_2020", 
                              "CRMCYMURD": "Student_MurdersIndex_2020", 
                              "CRMCYRAPE": "Student_RapesIndex_2020",
                              "CRMCYROBB": "Student_RobberiesIndex_2020", 
                              "CRMCYASST": "Student_AssaultsIndex_2020",
                              "CRMCYPROC": "Student_PropertyCrimesIndex_2020", 
                              "CRMCYBURG": "Student_BurglariesIndex_2020",
                              "CRMCYLARC": "Student_LarceniesIndex_2020", 
                              "CRMCYMVEH": "Student_VehicleTheftsIndex_2020",
                              "TOTPOP_CY": "Student_AreaPopulation_2020",
                              "Shape_Area": "Student_Area"}

In [50]:
# Read in the File
crime_df = pd.read_csv("/content/drive/MyDrive/Thesis/Codes/Combining Datasets/USA_Crime.csv", usecols=crime_file_columns)

# Create Two DataFrames Using the Crime Data
## One for the High School's Zip Code and the Other for the Student's Zip Code
hs_crime_df = crime_df.rename(columns=hs_crime_column_names)
student_crime_df = crime_df.rename(columns=student_crime_column_names)

##### <font color=darkturquoise>*Modify the Format of Columns, as Needed*</font>

In [51]:
# Fix the Zip Codes
## Convert the Zip Codes to a String and Add a Zero to the Front
hs_crime_df["HS_Crime_ZIP"] = hs_crime_df["HS_Crime_ZIP"].transform(lambda x: str("0" + str(x)))
student_crime_df["Student_Crime_ZIP"] = student_crime_df["Student_Crime_ZIP"].transform(lambda x: str("0" + str(x)))

# Categorize the Level of Crime for Each Crime Type
## High School's Area
hs_crime_df["HS_TotalCrimesCateg"] = hs_crime_df.apply(lambda x: categorize_crime(x.HS_CrimesIndex_2020), axis=1)
hs_crime_df["HS_PersonalCrimesCateg"] = hs_crime_df.apply(lambda x: categorize_crime(x.HS_PersonalCrimesIndex_2020), axis=1)
hs_crime_df["HS_MurdersCateg"] = hs_crime_df.apply(lambda x: categorize_crime(x.HS_MurdersIndex_2020), axis=1)
hs_crime_df["HS_RapesCateg"] = hs_crime_df.apply(lambda x: categorize_crime(x.HS_RapesIndex_2020), axis=1)
hs_crime_df["HS_RobberiesCateg"] = hs_crime_df.apply(lambda x: categorize_crime(x.HS_RobberiesIndex_2020), axis=1)
hs_crime_df["HS_AssaultsCateg"] = hs_crime_df.apply(lambda x: categorize_crime(x.HS_AssaultsIndex_2020), axis=1)
hs_crime_df["HS_PropertyCrimesCateg"] = hs_crime_df.apply(lambda x: categorize_crime(x.HS_PropertyCrimesIndex_2020), axis=1)
hs_crime_df["HS_BurglariesCateg"] = hs_crime_df.apply(lambda x: categorize_crime(x.HS_BurglariesIndex_2020), axis=1)
hs_crime_df["HS_LarceniesCateg"] = hs_crime_df.apply(lambda x: categorize_crime(x.HS_LarceniesIndex_2020), axis=1)
hs_crime_df["HS_VehicleTheftsCateg"] = hs_crime_df.apply(lambda x: categorize_crime(x.HS_VehicleTheftsIndex_2020), axis=1)

## Student's Area
student_crime_df["Student_TotalCrimesCateg"] = student_crime_df.apply(lambda x: categorize_crime(x.Student_CrimesIndex_2020), axis=1)
student_crime_df["Student_PersonalCrimesCateg"] = student_crime_df.apply(lambda x: categorize_crime(x.Student_PersonalCrimesIndex_2020), axis=1)
student_crime_df["Student_MurdersCateg"] = student_crime_df.apply(lambda x: categorize_crime(x.Student_MurdersIndex_2020), axis=1)
student_crime_df["Student_RapesCateg"] = student_crime_df.apply(lambda x: categorize_crime(x.Student_RapesIndex_2020), axis=1)
student_crime_df["Student_RobberiesCateg"] = student_crime_df.apply(lambda x: categorize_crime(x.Student_RobberiesIndex_2020), axis=1)
student_crime_df["Student_AssaultsCateg"] = student_crime_df.apply(lambda x: categorize_crime(x.Student_AssaultsIndex_2020), axis=1)
student_crime_df["Student_PropertyCrimesCateg"] = student_crime_df.apply(lambda x: categorize_crime(x.Student_PropertyCrimesIndex_2020), axis=1)
student_crime_df["Student_BurglariesCateg"] = student_crime_df.apply(lambda x: categorize_crime(x.Student_BurglariesIndex_2020), axis=1)
student_crime_df["Student_LarceniesCateg"] = student_crime_df.apply(lambda x: categorize_crime(x.Student_LarceniesIndex_2020), axis=1)
student_crime_df["Student_VehicleTheftsCateg"] = student_crime_df.apply(lambda x: categorize_crime(x.Student_VehicleTheftsIndex_2020), axis=1)

In [52]:
hs_crime_df

,HS_Crime_ZIP,HS_CrimesIndex_2020,HS_PersonalCrimesIndex_2020,HS_MurdersIndex_2020,HS_RapesIndex_2020,HS_RobberiesIndex_2020,HS_AssaultsIndex_2020,HS_PropertyCrimesIndex_2020,HS_BurglariesIndex_2020,HS_LarceniesIndex_2020,...,HS_TotalCrimesCateg,HS_PersonalCrimesCateg,HS_MurdersCateg,HS_RapesCateg,HS_RobberiesCateg,HS_AssaultsCateg,HS_PropertyCrimesCateg,HS_BurglariesCateg,HS_LarceniesCateg,HS_VehicleTheftsCateg
0,07001,46.0,22.0,15.0,12.0,41.0,17.0,50.0,37.0,54.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
1,07002,50.0,55.0,23.0,37.0,80.0,50.0,49.0,41.0,52.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
2,07003,60.0,34.0,21.0,7.0,74.0,24.0,64.0,47.0,66.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
3,07004,93.0,19.0,7.0,14.0,16.0,22.0,104.0,44.0,121.0,...,Low,Low,Low,Low,Low,Low,High,Low,High,Low
4,07005,34.0,20.0,9.0,34.0,21.0,17.0,36.0,27.0,40.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,08887,6.0,4.0,7.0,10.0,2.0,3.0,7.0,18.0,5.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
573,08889,28.0,10.0,26.0,9.0,6.0,12.0,31.0,33.0,33.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
574,08901,102.0,146.0,95.0,95.0,292.0,97.0,95.0,122.0,94.0,...,High,High,Low,Low,High,Low,Low,High,Low,Low
575,08902,65.0,36.0,17.0,25.0,52.0,31.0,69.0,59.0,76.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low


In [53]:
student_crime_df

,Student_Crime_ZIP,Student_CrimesIndex_2020,Student_PersonalCrimesIndex_2020,Student_MurdersIndex_2020,Student_RapesIndex_2020,Student_RobberiesIndex_2020,Student_AssaultsIndex_2020,Student_PropertyCrimesIndex_2020,Student_BurglariesIndex_2020,Student_LarceniesIndex_2020,...,Student_TotalCrimesCateg,Student_PersonalCrimesCateg,Student_MurdersCateg,Student_RapesCateg,Student_RobberiesCateg,Student_AssaultsCateg,Student_PropertyCrimesCateg,Student_BurglariesCateg,Student_LarceniesCateg,Student_VehicleTheftsCateg
0,07001,46.0,22.0,15.0,12.0,41.0,17.0,50.0,37.0,54.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
1,07002,50.0,55.0,23.0,37.0,80.0,50.0,49.0,41.0,52.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
2,07003,60.0,34.0,21.0,7.0,74.0,24.0,64.0,47.0,66.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
3,07004,93.0,19.0,7.0,14.0,16.0,22.0,104.0,44.0,121.0,...,Low,Low,Low,Low,Low,Low,High,Low,High,Low
4,07005,34.0,20.0,9.0,34.0,21.0,17.0,36.0,27.0,40.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,08887,6.0,4.0,7.0,10.0,2.0,3.0,7.0,18.0,5.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
573,08889,28.0,10.0,26.0,9.0,6.0,12.0,31.0,33.0,33.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
574,08901,102.0,146.0,95.0,95.0,292.0,97.0,95.0,122.0,94.0,...,High,High,Low,Low,High,Low,Low,High,Low,Low
575,08902,65.0,36.0,17.0,25.0,52.0,31.0,69.0,59.0,76.0,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low


In [54]:
print(hs_crime_df.info())
print()
print(student_crime_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   HS_Crime_ZIP                 577 non-null    object 
 1   HS_CrimesIndex_2020          574 non-null    float64
 2   HS_PersonalCrimesIndex_2020  574 non-null    float64
 3   HS_MurdersIndex_2020         574 non-null    float64
 4   HS_RapesIndex_2020           574 non-null    float64
 5   HS_RobberiesIndex_2020       574 non-null    float64
 6   HS_AssaultsIndex_2020        574 non-null    float64
 7   HS_PropertyCrimesIndex_2020  574 non-null    float64
 8   HS_BurglariesIndex_2020      574 non-null    float64
 9   HS_LarceniesIndex_2020       574 non-null    float64
 10  HS_VehicleTheftsIndex_2020   574 non-null    float64
 11  HS_AreaPopulation_2020       574 non-null    float64
 12  HS_Area                      577 non-null    float64
 13  HS_TotalCrimesCateg 

#### <font color=darkorange>Merge with `UB_AllSchools`</font>

##### <font color=lightsalmon>*For the High School's Area*</font>

In [55]:
# Merge with Upward Bound, High School, and College Data Using High School's Zip Code
UB_Schools_Crime = pd.merge(UB_AllSchools, hs_crime_df, how="left", left_on="HS_ZIP", right_on="HS_Crime_ZIP")

##### <font color=darkturquoise>*Check for Missing Values*</font>

In [56]:
print("Size of DataFrame: ", UB_Schools_Crime.shape)
print("Size of DataFrame after Dropping Null Values: ", UB_Schools_Crime.dropna().shape)

Size of DataFrame:  (146, 108)
Size of DataFrame after Dropping Null Values:  (146, 108)


In [57]:
# Remove the HS_Crime_ZIP Column
## This is No Longer Needed and is Redundant with HS_ZIP
UB_Schools_Crime.drop("HS_Crime_ZIP", axis=1, inplace=True)

##### <font color=lightsalmon>*For the Student's Area*</font>

In [58]:
# Merge with Upward Bound, High School, and College Data Using Student's Zip Code
final_df = pd.merge(UB_Schools_Crime, student_crime_df, how="left", left_on="Student_ZIP", right_on="Student_Crime_ZIP")

##### <font color=darkturquoise>*Check for Missing Values*</font>

In [59]:
print("Size of DataFrame: ", final_df.shape)
print("Size of DataFrame after Dropping Null Values: ", final_df.dropna().shape)

Size of DataFrame:  (146, 130)
Size of DataFrame after Dropping Null Values:  (146, 130)


In [60]:
# Remove the Student_Crime_ZIP Column
## This is No Longer Needed and is Redundant with Student_ZIP
final_df.drop("Student_Crime_ZIP", axis=1, inplace=True)

## <font color=royalblue>Compute the Distances</font>

#### <font color=darkviolet>Student's Distance to High School</font>

In [61]:
final_df["Distance_to_HS"] = final_df.apply(lambda x: get_distance(x.Student_Lat, x.Student_Long, x.HS_Lat, x.HS_Long), axis=1)

#### <font color=darkviolet>Student's Distance to College</font>

In [62]:
final_df["Distance_to_PS"] = final_df.apply(lambda x: get_distance(x.Student_Lat, x.Student_Long, x.PS_Lat, x.PS_Long), axis=1)

## <font color=royalblue>Write the Final Dataset to a .csv File</font>

##### <font color=darkturquoise>*Add a Hashtag Infront of the Two Zip Code Variables (Student and High School) to Use in Code #5*</font>

In [63]:
final_df["Student_ZIP"] = final_df.Student_ZIP.transform(lambda x: "#" + str(x))
final_df["HS_ZIP"] = final_df.HS_ZIP.transform(lambda x: "#" + str(x))

##### <font color=darkturquoise>*Drop the Unnecessary Columns*</font>

In [64]:
# Columns to Drop
drop_columns = ["DOB", "HS_SchoolID", "Date_EnteredUB", "StartGradeLevel", "EndGradeLevel", "GradStatus", "GraduationDate", 
                "Student_Lat", "Student_Long", "PS_DateEnrolled", "PS_Cohort", "PS_SchoolID", "PS_GradeLevel", "Cert_Date", 
                "AssocDeg_Date", "BachDeg_Date", "District", "HS_Lat", "HS_Long", "Phone", "PS_FullAddress", "PS_Lat", 
                "PS_Long", "PS_ZIP"]

final_df.drop(drop_columns, axis=1, inplace=True)
final_df

,Gender,Hispanic,AmerInd,Asian,Black,White,Hawaiian,Multinational,LimitedEnglish,Eligibility,...,Student_MurdersCateg,Student_RapesCateg,Student_RobberiesCateg,Student_AssaultsCateg,Student_PropertyCrimesCateg,Student_BurglariesCateg,Student_LarceniesCateg,Student_VehicleTheftsCateg,Distance_to_HS,Distance_to_PS
0,1,0,0,1,0,0,0,0,1,1,...,High,Low,High,High,High,High,Low,High,3.922676,19.596591
1,1,0,0,0,1,0,0,0,0,1,...,High,Low,High,High,Low,Low,Low,High,0.677924,2.130889
2,1,1,0,0,0,0,0,0,0,1,...,High,Low,High,Low,Low,High,Low,High,1.050270,111.509517
3,1,0,0,0,1,0,0,0,0,1,...,High,High,High,High,High,High,Low,High,2.263438,18.169620
4,1,0,0,0,1,0,0,0,0,2,...,High,High,High,High,High,High,Low,High,1.611439,3.422135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,0,0,0,0,1,0,0,0,0,1,...,High,Low,High,High,High,High,Low,High,21.119729,6.756697
142,1,1,0,0,0,1,0,1,0,1,...,High,Low,High,High,Low,High,Low,High,3.867340,18.598110
143,0,1,0,0,0,1,0,1,0,1,...,High,Low,High,High,High,High,Low,High,1.338182,17.132149
144,1,1,0,0,0,0,0,0,0,2,...,High,Low,High,High,Low,High,Low,High,20.062405,19.992918


In [ ]:
# Write the DataFrame to a .csv File
final_df.to_csv("/content/drive/MyDrive/Thesis/Codes/Preparation Datasets/cjb_thesis_dataset.csv", index_label="ID")